<a href="https://colab.research.google.com/github/balakumar-dataengineer/testrepo/blob/master/Pyspark_MapType_column.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Pyspark') \
                          .master('local') \
                          .getOrCreate()

In [12]:
data=[
    (1,'bala',{'primarySkill':'SQL','secondarySkill':'GCP'}),
    (2,'kumar',{'primarySkill':'Python','secondarySkill':'Pyspark'})
]

schema=['id','name','skills']

df=spark.createDataFrame(data,schema)
df.show(truncate=False)
df.printSchema()

+---+-----+---------------------------------------------------+
|id |name |skills                                             |
+---+-----+---------------------------------------------------+
|1  |bala |{secondarySkill -> GCP, primarySkill -> SQL}       |
|2  |kumar|{secondarySkill -> Pyspark, primarySkill -> Python}|
+---+-----+---------------------------------------------------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- skills: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [15]:
from pyspark.sql.functions import array

df1=df.withColumn('primarySkill',df.skills['primarySkill']) \
  .withColumn('secondarySkill',df.skills['secondarySkill']) \

df1.show(truncate=False)

df1.withColumn('array',array(df1.primarySkill,df1.secondarySkill)).show()


+---+-----+---------------------------------------------------+------------+--------------+
|id |name |skills                                             |primarySkill|secondarySkill|
+---+-----+---------------------------------------------------+------------+--------------+
|1  |bala |{secondarySkill -> GCP, primarySkill -> SQL}       |SQL         |GCP           |
|2  |kumar|{secondarySkill -> Pyspark, primarySkill -> Python}|Python      |Pyspark       |
+---+-----+---------------------------------------------------+------------+--------------+

+---+-----+--------------------+------------+--------------+-----------------+
| id| name|              skills|primarySkill|secondarySkill|            array|
+---+-----+--------------------+------------+--------------+-----------------+
|  1| bala|{secondarySkill -...|         SQL|           GCP|       [SQL, GCP]|
|  2|kumar|{secondarySkill -...|      Python|       Pyspark|[Python, Pyspark]|
+---+-----+--------------------+------------+-------

In [16]:
from pyspark.sql.types import StructField,StructType,StringType,IntegerType, MapType

schema1=StructType([
    StructField('id',IntegerType(),True),
    StructField('name',StringType(),True),
    StructField('skills',MapType(StringType(),StringType()),True)
])

df2=spark.createDataFrame(data,schema1)
df2.show(truncate=False)
df2.printSchema()

+---+-----+---------------------------------------------------+
|id |name |skills                                             |
+---+-----+---------------------------------------------------+
|1  |bala |{secondarySkill -> GCP, primarySkill -> SQL}       |
|2  |kumar|{secondarySkill -> Pyspark, primarySkill -> Python}|
+---+-----+---------------------------------------------------+

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- skills: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [17]:
df2.withColumn('primarySkill',df2.skills.getItem('primarySkill')) \
   .withColumn('secondarySkill',df2.skills.getItem('secondarySkill')) \
   .show()

+---+-----+--------------------+------------+--------------+
| id| name|              skills|primarySkill|secondarySkill|
+---+-----+--------------------+------------+--------------+
|  1| bala|{secondarySkill -...|         SQL|           GCP|
|  2|kumar|{secondarySkill -...|      Python|       Pyspark|
+---+-----+--------------------+------------+--------------+

